In [1]:
# a Classifier categize data into one or more of set of classes

In [2]:
!pip install opendatasets

In [3]:
import opendatasets as od
#{"username":"gnm3000","key":"3b8087f0208b7cc5ddc14cefc4132813"}

In [4]:
od.download("https://www.kaggle.com/paramaggarwal/fashion-product-images-small")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gnm3000
Your Kaggle Key: ··········


100%|██████████| 565M/565M [00:12<00:00, 45.8MB/s]


In [9]:
ls

images/  myntradataset/  styles.csv


In [16]:
import pandas as pd
df = pd.read_csv("styles.csv",error_bad_lines=False)

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [17]:
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [18]:
df["image"] = df.apply(lambda row: str(row["id"])+".jpg",axis=1)

In [19]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [20]:
df = df.sample(frac=1)
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
32558,23534,Women,Personal Care,Fragrance,Perfume and Body Mist,Silver,Spring,2017.0,Casual,DKNY Women Energise Perfume,23534.jpg
28668,11698,Men,Apparel,Topwear,Shirts,Blue,Fall,2011.0,Casual,Lee Men Check Blue Shirts,11698.jpg
29564,37558,Girls,Apparel,Bottomwear,Shorts,Pink,Summer,2012.0,Casual,Madagascar3 Girls Pink Shorts,37558.jpg
25026,57397,Women,Apparel,Innerwear,Briefs,Pink,Winter,2015.0,Casual,Amante Women Pink Briefs PGDI03,57397.jpg
39731,53126,Women,Apparel,Topwear,Tshirts,Blue,Summer,2012.0,Casual,Myntra Women Blue T-shirt,53126.jpg
...,...,...,...,...,...,...,...,...,...,...,...
22859,27827,Men,Accessories,Belts,Belts,Black,Summer,2013.0,Casual,Bulchee Men Black Leather Belt,27827.jpg
439,25188,Women,Accessories,Bags,Handbags,Brown,Winter,2015.0,Casual,Lino Perros Women Leatherite Brown Handbag,25188.jpg
25074,23216,Women,Apparel,Topwear,Kurtas,Blue,Fall,2011.0,Ethnic,Aneri Women Blue Julie Kurta,23216.jpg
31770,16210,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Polo Blue TShirt,16210.jpg


In [21]:
df = df.sample(frac=1).reset_index(drop=True)

In [22]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,59329,Unisex,Footwear,Shoes,Casual Shoes,Black,Summer,2012.0,Casual,Gliders Unisex Zoro Shoes,59329.jpg
1,48159,Women,Accessories,Wallets,Wallets,Silver,Winter,2016.0,Casual,Fossil Women Silver Wallet,48159.jpg
2,9187,Men,Apparel,Bottomwear,Jeans,Blue,Fall,2011.0,Casual,Spykar Men Style Essentials Spykar Basic Attir...,9187.jpg
3,29127,Women,Accessories,Eyewear,Sunglasses,Black,Winter,2016.0,Casual,Vogue Women Black Sunglasses,29127.jpg
4,13298,Men,Apparel,Bottomwear,Shorts,White,Fall,2011.0,Casual,Gini and Jony Men Solid White Shorts,13298.jpg


In [24]:
# preparing the ImageDataGenerator for training the model

from keras_preprocessing.image import ImageDataGenerator
batch_size = 256

image_generator = ImageDataGenerator(validation_split = 0.2)

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory="images",
    x_col="image",
    y_col="masterCategory",
    target_size = (60,80),
    batch_size = batch_size,
    subset="training"

)

validation_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory="images",
    x_col="image",
    y_col="masterCategory",
    target_size = (60,80),
    batch_size = batch_size,
    subset="validation"
)
classes = len(training_generator.class_indices)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 35536 validated image filenames belonging to 7 classes.
Found 8883 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 5 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [25]:
classes

7

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

In [28]:
classifier = Sequential()

classifier.add(Conv2D(32,(3,3),input_shape = (60,80,3),activation="relu"))
classifier.add(MaxPooling2D(pool_size=(3,3)))
classifier.add(Conv2D(32,(3,3),activation="relu"))
classifier.add(MaxPooling2D(pool_size=(3,3)))

classifier.add(Flatten())

classifier.add(Dense(units=32,activation="relu"))
classifier.add(Dense(units=64,activation="relu"))
classifier.add(Dense(units=128,activation="relu"))
classifier.add(Dense(units=256,activation="relu"))
classifier.add(Dense(units=512,activation="relu"))
classifier.add(Dense(units=7,activation="softmax"))
classifier.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])


In [31]:
# fitting the data in out NN


from math import ceil


classifier.fit_generator(
    generator=training_generator,
    steps_per_epoch=ceil(0.8*(df.shape[0]/batch_size)),
    validation_data=validation_generator,
    validation_steps = ceil(0.2*(df.shape[0]/batch_size)),
    epochs = 5,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  del sys.path[0]


Epoch 1/5
139/139 [==============================] - 34s 176ms/step - loss: 0.5334 - accuracy: 0.8363 - val_loss: 0.2301 - val_accuracy: 0.9256
Epoch 2/5
139/139 [==============================] - 23s 167ms/step - loss: 0.2139 - accuracy: 0.9319 - val_loss: 0.2033 - val_accuracy: 0.9429
Epoch 3/5
139/139 [==============================] - 23s 169ms/step - loss: 0.1608 - accuracy: 0.9504 - val_loss: 0.1850 - val_accuracy: 0.9465
Epoch 4/5
139/139 [==============================] - 23s 168ms/step - loss: 0.1385 - accuracy: 0.9575 - val_loss: 0.1497 - val_accuracy: 0.9553
Epoch 5/5
139/139 [==============================] - 23s 168ms/step - loss: 0.1223 - accuracy: 0.9631 - val_loss: 0.1636 - val_accuracy: 0.9475


In [33]:
loss,acc = classifier.evaluate_generator(validation_generator,steps=ceil(0.2*(df.size / batch_size)))
print(classifier.metrics_names[1])
print(acc*100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


accuracy
94.75402235984802


In [34]:
# saving model

classifier.save("model.h5")

In [35]:
# testing on our data

import numpy as np
from keras.preprocessing import image
filename = "15970.jpg"
from keras.models import load_model
new_model = load_model("model.h5")
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 58, 78, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 26, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 24, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 32)                4

In [38]:
test_image = image.load_img("images/"+filename,target_size=(60,80))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = new_model.predict(test_image)
print(result)

[[3.3436958e-03 9.9645412e-01 8.3141320e-05 1.0812044e-04 4.9048865e-08
  1.0314085e-05 6.0697454e-07]]


In [39]:
val = np.argmax(result)
print(val)

1


In [40]:
my_dict = training_generator.class_indices
key_list = list(my_dict.keys())
val_list = list(my_dict.values())
print(key_list[val])

Apparel


In [44]:
key_list

['Accessories',
 'Apparel',
 'Footwear',
 'Free Items',
 'Home',
 'Personal Care',
 'Sporting Goods']

In [48]:
# saving class ids

import pickle

filename1= "key_list"
filename = "val_list"
pickle.dump(key_list,open(filename1,"wb"))
pickle.dump(val_list,open(filename,"wb"))
